In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from datetime import datetime
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

from dataset import DataSet
from model_part import conv2d
from model_part import fc

In [2]:
flags = tf.app.flags
FLAGS = flags.FLAGS
today = datetime.strftime(datetime.now(), '%d%m%y')
flags.DEFINE_integer('batch_size', 8, 'Batch size')
flags.DEFINE_string('train_file', 'train.csv', 'Train file')
flags.DEFINE_string('gpu', '/gpu:0', 'GPU to run')
flags.DEFINE_integer('max_steps', 10000000, 'Max steps')
flags.DEFINE_boolean('log_device_placement', True,
                     'Log device placement')  # stores True by default
flags.DEFINE_string('train_dir', os.path.join(today, 'train'),
                    'Coarse directory')
flags.DEFINE_string('log_dir', os.path.join(today, 'logs'),
                    'Log directory')
flags.DEFINE_string('output_dir', os.path.join(today, 'output'),
                    'Output prediction directory')

In [25]:
from model_part import conv2d
from model_part import fc
def decoder_model(inputs, reuse=False, trainable=True, debug=False):
    # inputs.get_shape() = TensorShape([Dimension(8), Dimension(55), Dimension(74), Dimension(1)])
    if debug: print('%s \t\t\t%s' % (inputs.name, inputs.get_shape()))
    net = conv2d('coarse1', images, [11, 11, 3, 96], [96], [1, 4, 4, 1], padding='VALID', reuse=reuse, trainable=trainable)
    if debug: print('%s \t\t%s' % (net.name, net.get_shape()))
    net = tf.nn.max_pool(net, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='VALID', name='pool1')
    if debug: print('%s \t\t\t%s' % (net.name, net.get_shape()))
    net = conv2d('coarse2', net, [5, 5, 96, 256], [256], [1, 1, 1, 1], padding='VALID', reuse=reuse, trainable=trainable)
    if debug: print('%s \t\t%s' % (net.name, net.get_shape()))
    net = tf.nn.max_pool(net, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME', name='pool1')
    if debug: print('%s \t\t\t%s' % (net.name, net.get_shape()))
    net = conv2d('coarse3', net, [3, 3, 256, 384], [384], [1, 1, 1, 1], padding='VALID', reuse=reuse, trainable=trainable)
    if debug: print('%s \t\t%s' % (net.name, net.get_shape()))
    net = conv2d('coarse4', net, [3, 3, 384, 384], [384], [1, 1, 1, 1], padding='VALID', reuse=reuse, trainable=trainable)
    if debug: print('%s \t\t%s' % (net.name, net.get_shape()))
    net = conv2d('coarse5', net, [3, 3, 384, 256], [256], [1, 1, 1, 1], padding='VALID', reuse=reuse, trainable=trainable)
    if debug: print('%s \t\t%s' % (net.name, net.get_shape()))
    net = fc('coarse6', net, [6*10*256, 4096], [4096], reuse=reuse, trainable=trainable)
    if debug: print('%s \t\t%s' % (net.name, net.get_shape()))
    net = fc('coarse7', net, [4096, 6*10*256], [6*10*256], reuse=reuse, trainable=trainable)
    if debug: print('%s \t\t%s' % (net.name, net.get_shape()))
    net = tf.reshape(net, [-1, 6, 10, 256])
    if debug: print('%s \t\t\t%s' % (net.name, net.get_shape()))
    net = tf.image.resize_images(net, size=[8, 12], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    if debug: print('%s \t%s' % (net.name, net.get_shape()))
#     net = tf.pad(net, paddings=[[0,0], [1,1], [1,1], [0,0]], mode='SYMMETRIC')
#     if debug: print('%s \t\t\t%s' % (net.name, net.get_shape()))
    net = conv2d('up1', net, [3, 3, 256, 384], [384], [1, 1, 1, 1], padding='SAME', reuse=reuse, trainable=trainable)
    if debug: print('%s \t\t\t%s' % (net.name, net.get_shape()))
    net = tf.image.resize_images(net, size=[10, 14], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    if debug: print('%s \t%s' % (net.name, net.get_shape()))
    net = conv2d('up2', net, [3, 3, 384, 384], [384], [1, 1, 1, 1], padding='SAME', reuse=reuse, trainable=trainable)
    if debug: print('%s \t\t\t%s' % (net.name, net.get_shape()))
    net = tf.image.resize_images(net, size=[12, 16], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    if debug: print('%s \t%s' % (net.name, net.get_shape()))
    net = conv2d('up3', net, [3, 3, 384, 256], [256], [1, 1, 1, 1], padding='SAME', reuse=reuse, trainable=trainable)
    if debug: print('%s \t\t\t%s' % (net.name, net.get_shape()))
    net = tf.image.resize_images(net, size=[27, 36], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    if debug: print('%s \t%s' % (net.name, net.get_shape()))
    net = conv2d('up4', net, [3, 3, 256, 256], [256], [1, 1, 1, 1], padding='SAME', reuse=reuse, trainable=trainable)
    if debug: print('%s \t\t\t%s' % (net.name, net.get_shape()))
    net = tf.image.resize_images(net, size=[55, 74], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    if debug: print('%s \t%s' % (net.name, net.get_shape()))
    net = conv2d('up5', net, [5, 5, 256, 96], [96], [1, 1, 1, 1], padding='SAME', reuse=reuse, trainable=trainable)
    if debug: print('%s \t\t\t%s' % (net.name, net.get_shape()))
    net = tf.image.resize_images(net, size=[228, 304], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    if debug: print('%s \t%s' % (net.name, net.get_shape()))
    net = conv2d('up6', net, [11, 11, 96, 1], [1], [1, 1, 1, 1], padding='SAME', reuse=reuse, trainable=trainable)
    if debug: print('%s \t\t\t%s' % (net.name, net.get_shape()))
#     net = fc('up_fc1', net, [228*304, 228*304], [228*304], reuse=reuse, trainable=trainable)
#     if debug: print('%s \t\t\t%s' % (net.name, net.get_shape()))
#     net = fc('up_fc2', net, [228*304, 228*304], [228*304], reuse=reuse, trainable=trainable)
#     if debug: print('%s \t\t\t%s' % (net.name, net.get_shape()))
#     net = tf.reshape(net, [-1, 228, 304, 1])
#     if debug: print('%s \t\t\t%s' % (net.name, net.get_shape()))

In [26]:
with tf.Graph().as_default():
    global_step = tf.Variable(0, trainable=False)
    dataset = DataSet(FLAGS.batch_size)
    images, depths, invalid_depths = dataset.csv_inputs(FLAGS.train_file, target_size=[228, 304])
    keep_conv = tf.placeholder(tf.float32)
    keep_hidden = tf.placeholder(tf.float32)
    encoder_model = decoder_model(images, keep_conv, trainable=False, debug=True)

batch:0 			(8, 228, 304, 3)
coarse1/coarse1:0 		(8, 55, 74, 96)
pool1:0 			(8, 27, 36, 96)
coarse2/coarse2:0 		(8, 23, 32, 256)
pool1_1:0 			(8, 12, 16, 256)
coarse3/coarse3:0 		(8, 10, 14, 384)
coarse4/coarse4:0 		(8, 8, 12, 384)
coarse5/coarse5:0 		(8, 6, 10, 256)
coarse6/coarse6:0 		(8, 4096)
coarse7/coarse7:0 		(8, 15360)
Reshape:0 			(8, 6, 10, 256)
ResizeNearestNeighbor:0 	(8, 8, 12, 256)
up1/up1:0 			(8, 8, 12, 384)
ResizeNearestNeighbor_1:0 	(8, 10, 14, 384)
up2/up2:0 			(8, 10, 14, 384)
ResizeNearestNeighbor_2:0 	(8, 12, 16, 384)
up3/up3:0 			(8, 12, 16, 256)
ResizeNearestNeighbor_3:0 	(8, 27, 36, 256)
up4/up4:0 			(8, 27, 36, 256)
ResizeNearestNeighbor_4:0 	(8, 55, 74, 256)
up5/up5:0 			(8, 55, 74, 96)
ResizeNearestNeighbor_5:0 	(8, 228, 304, 96)
up6/up6:0 			(8, 228, 304, 1)
